# Homework 2 for 18.337, September 2016
    
Due at 11:59pm on **Monday 26 September 2016**.

Please submit the homework to `mit.class.18337@gmail.com`

### 1. Create your own matrix type

In this question, we will explore Julia's type system in more detail, by defining a new type of matrix in three different ways.

A symmetric [arrow (or arrowhead) matrix](https://en.wikipedia.org/wiki/Arrowhead_matrix) is a matrix whose elements are non-zero only on the diagonal and in the first row and (symmetrically) the first column:

<img src="arrowhead.png", width=250>

(i) Use Julia's sparse matrix capabilities to define a symmetric arrow matrix: define vectors `I` and `J` containing the row ($i$) and column ($j$) coordinates of the non-zero entries, and a vector `V` of the corresponding values. Create the matrix with `sparse(I, J, V)`. [Note: be careful not to define the upper left entry more than once.]

Fix a (largeish) arrow matrix that you will use throughout the question, and time how long the power method takes.

(ii) Implement a new Julia type, `SymArrowFloat`, that contains two vectors of `Float64`: 1. the diagonal entries; and 2. the first row or column (minus the first entry).

Write the following functions acting on this type:
- `full`, that creates a standard Julia matrix with the same contents
- `+` for adding two arrow matrices
- `*` for matrix-vector multiplication of a `SymArrowFloat` with a vector of `Float64`s
- `show` to display the matrix in a clear way. 
[Import `Base.show` and then define `show(io::IO, A::SymArrowFloat)`.]

Write tests to make sure that `*` works (using the `Base.Test` testing framework, as in homework 1).

Find the largest eigenvalue of a symmetric arrow matrix using the `power_method` code, and check that it is correct. 

(iii) Now suppose that we need an arrow matrix with elements that are rationals, or complex numbers. We could define separate, new types for each of those possibilities (`SymArrowRational`, `SymArrowComplex`, etc.). However, Julia provides an alternative that provides, once more, for **generic code**:

To make a new type `SymArrow` that can contain elements of an **arbitrary type** `T`, such as rationals or complex numbers, we use the following syntax:

In [1]:
type SymArrow{T}
    diag::Vector{T}
    first_row::Vector{T}  # without first entry
end

Here, `T` is a **type parameter**: we are defining a **template** for a new type that will contain elements of type `T`. 

Redefine `full`, `+` and `*`. These must also have type parameters, for example the signature of the `+` method will be

    +{T}(A::SymArrow{T}, B::SymArrow{T}) = ⋯
    
Here, `T` can just be thought of as another parameter of the function.

Make sure that you can create objects of type `SymArrow` with 1. rational; 2. complex and 3. `BigFloat` elements.

(iv) An alternative way to define such a type is to use some of the built-in machinery that Julia provides for arrays. In order to do so, we will make a new `SymArrow2` type, which we will declare to be a **subtype** (`<:`) of the abstract type `AbstractMatrix`, as follows:

In [5]:
type SymArrow2{T} <: AbstractMatrix{T}
    diag::Vector{T}
    first_row::Vector{T}  # without first entry
end

LoadError: LoadError: invalid redefinition of constant SymArrow2
while loading In[5], in expression starting on line 1

[For a detailed discussion of parametric types, see the [Julia manual](http://docs.julialang.org/en/release-0.5/manual/types/#man-parametric-types) and the tutorial [notebooks](https://github.com/dpsanders/intermediate_julia) and [video](https://www.youtube.com/watch?v=rAxzR7lMGDM).]

This will allow us to use many so-called "fall-back methods" that Julia provides, defined for types that to **automatically obtain functionality**, without explicitly providing it ourselves, as we had to in question (ii). **However, this may lead to less efficient code**, since the generic versions will not take advantage of the structure.

First, try using `*` of a `SymArrow2` and a vector. What happens? We see that in order to use the Julia-provided functionality, we must first define two methods for objects `A` of type `SymArrow2`:

- `size(A)`, which returns, as a tuple, the size of the array (matrix) in each direction
- `getindex(A, i, j)`, which defines what is returned when we query `A[i, j]`, i.e. access to the element at position `(i, j)`.

This is done as follows; fill in the details for `getindex`:

In [6]:
import Base: size, getindex

size{T}(A::SymArrow2{T}) = (length(A.diag), length(A.diag))

function getindex{T}(A::SymArrow2{T}, i, j) 
    
    i == j  ## fill this case in
    i == 1  ## fill this case in
    j == 1  ## fill this case in

    return zero(T)  # otherwise return zero of type T
end
        

getindex (generic function with 123 methods)

Now check that `full` and `*` work **automatically** for this type, having defined only `size` and `getindex`.

Time the power method for each version of the type. Which is fastest?

## 2. Parallel prefix algorithm

Show that the parallel prefix algorithm works through a recursive argument.
Specifically:
1. the first step performs the operation on odd-even pairs;
2. the second step is a **recursive** prefix algorithm;
3. the third step fills in the missing numbers.

We want you to understand this by whatever means will satisfy you.
This can be done via a computational experiment or mathematical proof.